# Shortest Distance from All Buildings

You want to build a house on an empty land which reaches all buildings in the shortest amount of distance. You can only move up, down, left and right. You are given a 2D grid of values 0, 1 or 2, where:

Each 0 marks an empty land which you can pass by freely.
Each 1 marks a building which you cannot pass through.
Each 2 marks an obstacle which you cannot pass through.
```
Example:

Input: [[1,0,2,0,1],[0,0,0,0,0],[0,0,1,0,0]]

1 - 0 - 2 - 0 - 1
|   |   |   |   |
0 - 0 - 0 - 0 - 0
|   |   |   |   |
0 - 0 - 1 - 0 - 0

Output: 7 
```

Explanation: Given three buildings at (0,0), (0,4), (2,2), and an obstacle at (0,2),
             the point (1,2) is an ideal empty land to build a house, as the total 
             travel distance of 3+3+1=7 is minimal. So return 7.
Note:
There will be at least one building. If it is not possible to build such house according to the above rules, return -1.

## Communication
**DPS approach** We could approach this problem by first interpreting the data and then running dfs to find the best empty land to build a house. When we interpret the data, we want to iterate over every node to create a list of buildings, list of empty land, and list of obstacles. We want the list of buildings when we're running dfs on the graph and we could identify if a building is found or if all buildings are discovered before we run the dfs on a new node. We store the list of empty lands to iterate over them to find the minimum distance to all the buildings. Finally, we have the obstacle list to avoid iterating over nodes that we cannot pass or are obstacles. To process the data we want to run dfs on every empty land using the empty lands list. When doing so, for every new iteration of empty land, we would maintain two local variables, a visited list and a distance to building list. The visited list would terminate the iteration of duplicate nodes and avoid unnecessary dfs processes. The building list would store the distances to each building from the starting point of the empty land. After each iteration of the empty land, we want to compare with the previous total distance and store the minimum distance between the previous and current. To find the next node to visit, we want to visit all possible nodes that have not been visited yet, are not buildings, and are not obstacles. We could use our building list and obstacle list to processing these nodes. The time complexity of this algorithm is $O(n^2)$ because for every empty land, we run dfs to check the path and distance to all buildings. The space complexity is $O(n)$ since n is the number of nodes we are storing in the visited list.

In [26]:
## Coding
class Solution(object):
    def shortestDistance(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        # gather data
        buildings = set()
        empty_lands = set()
        for row in range(len(grid)):
            for col in range(len(grid[0])):
                if grid[row][col] == 0:
                    empty_lands.add((row,col))
                elif grid[row][col] == 1:
                    buildings.add((row,col))
        # process data
        mindist = float('inf')
        for el_x, el_y in empty_lands:
            dist = 0
            for b_x, b_y in buildings:
                def dfs(i, j, k, l, dist):
                    # return intmax if indexes are out of bounds or empty lands are already visited
                    if i < 0 or i >= len(grid) or j < 0 or j >= len(grid[0]) or grid[i][j] == -1:
                        return float('inf')
                    # return building distance if building is discovered
                    elif i == k and j == l:
                        return dist
                    # return intmax if grid cell is not specified building or obstacle
                    elif grid[i][j] == 1 or grid[i][j] == 2:
                        return float('inf')
                    # store empty land cell value in tempval to represent already visited
                    # empty land cells with negative values
                    tempval = grid[i][j]
                    grid[i][j] = -1
                    # calculate the minimum path
                    dist = min(dfs(i-1,j,k,l,dist+1),
                               dfs(i+1,j,k,l,dist+1),
                               dfs(i,j-1,k,l,dist+1),
                               dfs(i,j+1,k,l,dist+1))
                    # restore empty land cell value
                    grid[i][j] = tempval
                    return dist
                tempdist = dfs(el_x, el_y, b_x, b_y, 0)
                if tempdist == float('inf'):
                    dist = 0
                    break
                dist += tempdist
            if dist != 0 and dist < mindist:
                mindist = dist
        return -1 if mindist == float('inf') else mindist
    def unit_tests(self):
        test_cases = [
            [[[1,0,2,0,1],[0,0,0,0,0],[0,0,1,0,0]], 7],
            [[[1,1,1,1,1,0],[0,0,0,0,0,1],[0,1,1,0,0,1],[1,0,0,1,0,1],[1,0,1,0,0,1],[1,0,0,0,0,1],[0,1,1,1,1,0]], 88]
        ]
        for index, tc in enumerate(test_cases):
            output = self.shortestDistance(tc[0])
            print('output: {0}'.format(output))
            assert output == tc[1], 'test#{0} failed'.format(index)
            print('test#{0} passed'.format(index))
Solution().unit_tests()

output: 7
test#0 passed
output: 88
test#1 passed


## Communication
**BFS Approach** In our previous approach with DFS, we have redundant processes that we would not need to process with BFS. If we shift towards the BFS appraoch, we're adding all adjacent cells to our queue, therefore we would have the shortest paths to all buildings without iterating over redundant cells like in the DPS. The problem with DFS is that it is not able to identify the shortest path to all buildings in one pass, we need to specify which building the dfs recursion is looking for. In our bfs approach, for each empty cell, if it can reach all buildings, we calculate sum of shortest distance from it to each building. This process takes $O(n^2)$ where $n$ is the number of cells in the grid. Since shortest distance is a transitive property, for each building (x,y) we calculate the shortest distance from (x,y) to each empty cell. Then we find the empty cell which can reach all buildings with minimum sum of distance to all buildings. This process takes $O(n)$ where $n$ is again the number of cells.

In [28]:
## Coding
## Coding
class Solution(object):
    def shortestDistance(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        # If it cannot build building, return -1
        if not grid:
            return -1
        rows, cols = len(grid), len(grid[0])
        reach = [[0 for _ in range(cols)] for _ in range(rows)]
        dist = [[0 for _ in range(cols)] for _ in range(rows)]
    
        def bfs(i, j, cnt):
            queue = [(i,j,0)]
            while queue:
                r, c, step = queue.pop(0)
                for dr, dc in ((0,1), (0,-1), (1,0), (-1,0)):
                    nr, nc = dr + r, dc + c
                    if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == 0 and reach[nr][nc] == cnt:
                        dist[nr][nc] += step + 1
                        reach[nr][nc] += 1
                        queue.append((nr, nc, step + 1))
                step += 1
    
        cnt = 0
        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == 1:
                    # For a building, calculate shortest distance to each empty cell by BFS
                    bfs(i, j, cnt)
                    cnt += 1
    
        ans = float('inf')
        for i in range(rows):
            for j in range(cols):
                # Get min value from distance if cell is empty and can reach all other buildings
                if grid[i][j] == 0 and reach[i][j] == cnt:
                    ans = min(ans, dist[i][j])
    
        return ans if ans < float('inf') else -1
    
    def unit_tests(self):
        test_cases = [
            [[[1,0,2,0,1],[0,0,0,0,0],[0,0,1,0,0]], 7],
            [[[1,1,1,1,1,0],[0,0,0,0,0,1],[0,1,1,0,0,1],[1,0,0,1,0,1],[1,0,1,0,0,1],[1,0,0,0,0,1],[0,1,1,1,1,0]], 88]
        ]
        for index, tc in enumerate(test_cases):
            output = self.shortestDistance(tc[0])
            print('output: {0}'.format(output))
            assert output == tc[1], 'test#{0} failed'.format(index)
            print('test#{0} passed'.format(index))
Solution().unit_tests()

output: 7
test#0 passed
output: 88
test#1 passed


## Reference
- [Leetcode](https://leetcode.com/problems/shortest-distance-from-all-buildings/)